**The code below to create the 4x4 mapping should create a positive mapping**

In [ ]:
# This code was created by Darshini Rajamani with editing from Luke

import random;

def get_random_number():
    return random.randint(-9, 9)

# Setting the requirements for Positive Semi-Definite matrices
def is_valid(matrix, rows, cols, row, col, num):
    # initial assigment to check validation
    matrix[row][col] = num

    # Check row condition (check in E(2,2))
    # sum of the first two equals the sum of the last two in each row
    if col == 3:
        if sum(matrix[row][:2]) != matrix[row][2] + num:
            matrix[row][col] = None
            return False

    # Check column condition (linearity check)
    # sum of the 1st and 2nd rows equal the 3rd and 4th
    if row == 3:
        if matrix[0][col] + matrix[1][col] != matrix[2][col] + num:
            matrix[row][col] = None
            return False

    # Positivity check
    # Sum of any element added to any other element is positive
    if col == 3:
        matrix[row][col] = num
        if matrix[row][0] + matrix[row][2] < 0:
            matrix[row][col] = None
            return False
        if matrix[row][0] + matrix[row][3] < 0:
            matrix[row][col] = None
            return False
        if matrix[row][1] + matrix[row][2] < 0:
            matrix[row][col] = None
            return False
        if matrix[row][1] + matrix[row][3] < 0:
            matrix[row][col] = None
            return False

    matrix[row][col] = None
    return True

# Creating the matrices
def create_matrix(rows, cols):
    matrix = [[None for _ in range(cols)] for _ in range(rows)] #has every element in the matrix as 'none'

    backtrackFailedAttempts = 0
    def backtrack(row, col):  # to check for complete and valid solution
        nonlocal backtrackFailedAttempts
        if backtrackFailedAttempts > 1000:
            return False

        if row == rows:
            return True

        nums = [i for i in range(-9, 10)]  # range [-9, 10)
        random.shuffle(nums)

        for num in nums:
            if is_valid(matrix, rows, cols, row, col, num):
                matrix[row][col] = num

                next_row = row
                next_col = col + 1
                if next_col == cols:
                    next_row += 1
                    next_col = 0

                if backtrack(next_row, next_col):
                    return True

        matrix[row][col] = None
        backtrackFailedAttempts = backtrackFailedAttempts + 1
        return False

    backtrack(0, 0)  # recursion

    return matrix

In [ ]:
# this code was created by Luke Luschwitz (mainly) and Karim with influence from Abbas and Darshini

from scipy.optimize import linprog
import numpy as np

numberOfMappingsToCreate = 64000
#######################
### Create Mappings ###
#######################
listOfMappings = []
for i in range(numberOfMappingsToCreate):
  rows = 4
  cols = 4

  isAValidMatrix = False

  while not isAValidMatrix:
    isAValidMatrix = True
    matrix = create_matrix(rows, cols)
    for row in matrix:
      for element in row:
        if element is None:
          isAValidMatrix = False

  # matrix[1], matrix[3] = matrix[3], matrix[1]

  listOfMappings.append(matrix)


###########################
# Check for Extendability #
###########################
extendableMappings = []
nonExtendableMappings = []

for matrix in listOfMappings:
  mat = [1, 1, 1, 1]
  for_A_ub = [[1,0,0,0],
            [0,1,0,0],
            [0,0,1,0],
            [0,0,0,1],
            [-1,0,0,0],
            [0,-1,0,0],
            [0,0,-1,0],
            [0,0,0,-1],]
  solve_for = [[min(matrix[0][2], matrix[0][3])],
             [min(matrix[1][2], matrix[1][3])],
             [min(matrix[2][2], matrix[2][3])],
             [min(matrix[3][2], matrix[3][3])],
             [-max(-matrix[0][0], -matrix[0][1])],
             [-max(-matrix[1][0], -matrix[1][1])],
             [-max(-matrix[2][0], -matrix[2][1])],
             [-max(-matrix[3][0], -matrix[3][1])]]
  for_A_eq = [[1,1,-1,-1]]
  for_b_eq = [0]

  result = linprog(c=mat, A_ub = for_A_ub, b_ub = solve_for, A_eq = for_A_eq, b_eq = for_b_eq, bounds = None)
  for row in matrix:
    print(row)
  if (result.success):
    extendableMappings.append(matrix)
    print("Extendable")
  else:
    nonExtendableMappings.append(matrix)
    print("Not extendable")
  print()

# Save the sample data to a file
np.save('/content/extendableMappings.npy', extendableMappings)
np.save('/content/nonExtendableMappings.npy', nonExtendableMappings)

In [15]:
# Import saved extendable and nonextendable mappings
import numpy as np
extendableMappings = np.load('/content/extendableMappings.npy')
nonExtendableMappings = np.load('/content/nonExtendableMappings.npy')
print(f"len(extendableMappings): {len(extendableMappings)}")
print(f"len(nonExtendableMappings): {len(nonExtendableMappings)}")

len(extendableMappings): 77462
len(nonExtendableMappings): 22538


In [ ]:
np.save('/content/extendableMappings.npy', extendableMappings)
np.save('/content/nonExtendableMappings.npy', nonExtendableMappings)

In [ ]:
# CODE BLOCK NOT IN USE

# Combine mappings that are stored in separate files
import numpy as np
extendableMappingsEXTRA = np.load('/content/extendableMappingsMORE.npy')
nonExtendableMappingsEXTRA = np.load('/content/nonExtendableMappingsMORE.npy')

extendableMappings = np.concatenate((extendableMappings, extendableMappingsEXTRA))
nonExtendableMappings = np.concatenate((nonExtendableMappings, nonExtendableMappingsEXTRA))

In [ ]:
# All the code below was made by Luke with slight influence from Abbas

import numpy as np

# This function returns the matrix B that is farthest away from the matrices in setA
def farthestBFromA(setB, setA):
  totalDistancesFromA = []
  # iterate through each B matrix
  for B in setB:
    currentBTotalDistance = 0
    # accumulate sum of distances from A to B (distance squared)
    for A in setA:
      currentBTotalDistance = currentBTotalDistance + (np.linalg.norm(np.subtract(B, A)) ** 2)
    totalDistancesFromA.append(currentBTotalDistance)

  orderedBsByDistance = []
  for i in range(len(totalDistancesFromA)):
    index = np.argmax(totalDistancesFromA)
    orderedBsByDistance.append(setB[index])
    totalDistancesFromA[index] = -np.inf
  return orderedBsByDistance


farthestBs = farthestBFromA(nonExtendableMappings, extendableMappings)

for i in range(len(farthestBs[0:3])):
  for row in farthestBs[i]:
      print(row)
  print(f"{i}th farthest from the extendables")
  print()

In [ ]:
# Save the farthestBs to a file
np.save('/content/farthestBs.npy', farthestBs)

In [25]:
# Load the saved farthestBs
farthestBs = np.load('/content/farthestBsMORE.npy')
print(f"len(farthestBs): {len(farthestBs)}")

len(farthestBs): 14380


In [22]:
# Find a classifier that linearly separates the extendable matrices from the nonextendable matrices using an SVM

import numpy as np
from sklearn import svm
from sklearn.metrics import accuracy_score

def CreateClassifier(farthestBIndex):
  # Declare global variables
  global extendableMappings
  global nonExtendableMappings
  global farthestBs

  # Ensure everything is a numpy array
  extendableMappings = np.array(extendableMappings)
  nonExtendableMappings = np.array(nonExtendableMappings)
  farthestBs = np.array(farthestBs)

  # Generate input by adding the farthestB matrix to the list of extendableMappings
  features = np.concatenate((
      extendableMappings,
      [farthestBs[farthestBIndex]],
      [extendableMappings[0]/1000],
      [farthestBs[farthestBIndex]/1000],
      [np.zeros((4, 4))]
      ))
  # print(f"len(features): {len(features)}")

  # Flatten the matrices into 1D arrays
  features = features.reshape(len(features), -1)

  # Generate labels (0 or 1)
  labels = [0]*len(extendableMappings) + [1,0,1,1]
  # print(f"len(labels): {len(labels)}")
  # print(f"labels: {labels}")

  # Create and train a support vector classifier
  model = svm.SVC(kernel='linear', C=1e10, coef0=0.0, tol=1e-5)
  model.fit(features, labels)

  # Set the bias (intercept) to be zero
  model.intercept_ = [0.0]

  # Make predictions on the test set
  predictions = model.predict(features)

  # Evaluate the accuracy of the model
  accuracy = accuracy_score(labels, predictions)
  # print(f"Accuracy: {accuracy}")

  # Get the coefficients (weights) of the hyperplane
  # coefficients = model.coef_.reshape(4,4)
  coefficients = model.coef_

  # Intercept of the hyperplane
  intercept = model.intercept_

  # print("Coefficients:\n", coefficients) #the matrix that we get from the ML program
  # print("Intercept:", intercept)

  return coefficients


In [27]:
# Tweaks the classifier slightly to find a 'nicer' classifier with whole numbers and low significant digits

def RoundClassifier(classifier):
  # Make sure classifier is a numpy array
  classifier = np.array(classifier)

  # Make a copy of the classifier
  c = classifier.copy()

  return np.round(c/10) # this is same as coefficients, but times 10 and rounded to integers

In [23]:
# Test if the input classifier correctly classifies all the generated sample data
def CheckClassifierAgainstSamples(classifier):
  # Combine all the samples into one array
  allMappings = np.concatenate((extendableMappings, nonExtendableMappings), axis=0)
  allMappings = allMappings.reshape(len(allMappings), -1)

  # Initialize the total counting variables
  totalMappingsInClass1 = 0
  totalMappingsInClass2 = 0
  classifiedMappingResults = []

  # Classify each sample by calculating the dot product of the classifier and each sample
  for i in range(len(allMappings)):
    dotProduct = np.dot(classifier, allMappings[i])
    dotProduct = np.sum(dotProduct)
    dotProductNormalized = dotProduct/(np.linalg.norm(classifier) * np.linalg.norm(allMappings[i]))
    extendableOrNot = "E" if i<len(extendableMappings) else "N"
    if dotProduct <= 0:
      # Supposed to be Extendable
      # print(f"{extendableOrNot} {dotProduct} (class 1)")
      classifiedMappingResults.append((allMappings[i], extendableOrNot, 1, i, dotProduct, dotProductNormalized))
      totalMappingsInClass1 += 1
    else:
      # Supposed to be nonExtendable
      # print(f"{extendableOrNot} {dotProduct} (class 2)")
      classifiedMappingResults.append((allMappings[i], extendableOrNot, 2, i, dotProduct, dotProductNormalized))
      totalMappingsInClass2 += 1

  # Check if samples were misclassified
  largestMisclassification = 0 # the largest dot product of a misclassified sample
  isGoodClassifier = True
  for mapping, extendability, _class, mapID, dotProduct, dotProductNormalized in classifiedMappingResults:
    if (extendability == "E" and _class == 2):
      # problemMappings = np.concatenate((problemMappings, [mapping]), axis=0)
      # print("Bad classifier, extendableMapping in class 2")
      # print(mapping.reshape(4,-1), extendability, _class, mapID, dotProduct, dotProductNormalized)
      isGoodClassifier = False
      if dotProductNormalized > largestMisclassification:
        largestMisclassification = dotProductNormalized
      # break
  if totalMappingsInClass1 < 1:
    # print("Bad classifier, no mappings in class 1")
    isGoodClassifier = False
  if totalMappingsInClass2 < 1:
    # print("Bad classifier, no mappings in class 2")
    isGoodClassifier = False

  # print(f"totalMappingsInClass1: {totalMappingsInClass1}")
  # print(f"totalMappingsInClass2: {totalMappingsInClass2}")
  # print(f"Largest misclassification: {largestMisclassification}")

  return isGoodClassifier


In [19]:
# Check that the found classifier is a true classifier using linear programming
from scipy.optimize import linprog
import numpy as np

def CheckIfTrueClassifier(classifier):
  classifier = np.array(classifier)

  c = -classifier.reshape(1, 16)

  c = np.concatenate((c, np.zeros((1, 4))), axis=1) # Add 4 dummy variables to check for extendability

  for_A_ub = [[-1, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], # positivity check
              [-1, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], # positivity check
              [0, -1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], # positivity check
              [0, -1, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], # positivity check
              [0, 0, 0, 0, -1, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], # positivity check
              [0, 0, 0, 0, -1, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], # positivity check
              [0, 0, 0, 0, 0, -1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], # positivity check
              [0, 0, 0, 0, 0, -1, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], # positivity check
              [0, 0, 0, 0, 0, 0, 0, 0, -1, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0], # positivity check
              [0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0], # positivity check
              [0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0], # positivity check
              [0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0], # positivity check
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, -1, 0, 0, 0, 0, 0], # positivity check
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, -1, 0, 0, 0, 0], # positivity check
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, 0, 0, 0, 0, 0], # positivity check
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, -1, 0, 0, 0, 0], # positivity check

              [-1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0], # extendability check
              [0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0], # extendability check
              [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], # extendability check
              [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], # extendability check
              [0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0], # extendability check
              [0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0], # extendability check
              [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], # extendability check
              [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], # extendability check
              [0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0], # extendability check
              [0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0], # extendability check
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0], # extendability check
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0], # extendability check
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, -1], # extendability check
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, -1], # extendability check
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1], # extendability check
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1]] # extendability check

  for_b_ub = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

  for_A_eq = [[1, 0, 0, 0, 1, 0, 0, 0, -1, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0], # linearity check
              [0, 1, 0, 0, 0, 1, 0, 0, 0, -1, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0], # linearity check
              [0, 0, 1, 0, 0, 0, 1, 0, 0, 0, -1, 0, 0, 0, -1, 0, 0, 0, 0, 0], # linearity check
              [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, -1, 0, 0, 0, -1, 0, 0, 0, 0], # linearity check
              [1, 1, -1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], # E(2,2) check
              [0, 0, 0, 0, 1, 1, -1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], # E(2,2) check
              [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, -1, -1, 0, 0, 0, 0, 0, 0, 0, 0], # E(2,2) check
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, -1, -1, 0, 0, 0, 0], # E(2,2) check
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, -1, -1]] # extendability check
  for_b_eq = [0, 0, 0, 0, 0, 0, 0, 0, 0]

  return linprog(c=c, A_ub=for_A_ub, b_ub=for_b_ub, A_eq=for_A_eq, b_eq=for_b_eq, bounds=(None, None))


In [40]:
classifier = [-2, -3, 2, -7,
              -2, -3, 0, -5,
              1, -1, -1, 1,
              2, -12, -4, -6]

print(CheckIfTrueClassifier(classifier))

print()

print(CheckClassifierAgainstSamples(classifier))

       message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
       success: True
        status: 0
           fun: 0.0
             x: [-0.000e+00 -0.000e+00 ... -0.000e+00 -0.000e+00]
           nit: 24
         lower:  residual: [       inf        inf ...        inf        inf]
                marginals: [ 0.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]
         upper:  residual: [       inf        inf ...        inf        inf]
                marginals: [ 0.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]
         eqlin:  residual: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00
                             0.000e+00  0.000e+00  0.000e+00  0.000e+00
                             0.000e+00]
                marginals: [ 2.000e+00 -1.000e+00 -1.000e+00  2.000e+00
                            -1.000e+00 -0.000e+00 -0.000e+00 -0.000e+00
                            -0.000e+00]
       ineqlin:  residual: [ 0.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]
                margina

In [28]:
# Find a classifier that linearly separates the extendable matrices from the nonextendable matrices using an SVM

import numpy as np
from sklearn import svm
from sklearn.metrics import accuracy_score

# Initialize true classifier array
trueClassifiers = np.zeros((1,16))

# Create classifiers and test if they are true classifiers
for i in range(1000):
  # Create a classifier using the ith farthest nonExtendable mapping
  classifier = CreateClassifier(i)

  # Round the classifier (elementwize)
  classifier = RoundClassifier(classifier)

  # Test if the classifier is a true classifier
  result = CheckIfTrueClassifier(classifier)

  if result.success:
    trueClassifiers = np.concatenate((trueClassifiers, classifier), axis=0)

trueClassifiers = trueClassifiers[1:]
print(f"len(trueClassifiers): {len(trueClassifiers)}\n")
print(trueClassifiers)

len(trueClassifiers): 1000

[[ -1.  -5.   1. ...  -8.   2. -12.]
 [  0.  -5.  -0. ...  -8.   1.  -9.]
 [ -0.  -6.  -8. ... -10. -12.   1.]
 ...
 [ -3.  -1.   1. ...  -0.  -4.  -7.]
 [ -1.  -5.   1. ...  -1.  -6.  -4.]
 [ -0.  -2.   0. ...  -1.  -3.  -5.]]


In [29]:
# Check if the classifiers correctly classify on all the sample data

trueClassifiersGood = np.zeros((1,16))
trueClassifiersBad = np.zeros((1,16))

allMappings = np.concatenate((extendableMappings, nonExtendableMappings), axis=0)
allMappings = allMappings.reshape(len(allMappings), -1)

for k in range(len(trueClassifiers)):
  totalMappingsInClass1 = 0
  totalMappingsInClass2 = 0
  classifiedMappingResults = []

  for i in range(len(allMappings)):
    dotProduct = np.dot(trueClassifiers[k], allMappings[i])
    dotProduct = np.sum(dotProduct)
    extendableOrNot = "E" if i<len(extendableMappings) else "N"
    if dotProduct <= 0:
      # extendable or nonextendable
      # print(f"{extendableOrNot} {dotProduct} (class 1)")
      classifiedMappingResults.append((allMappings[i], extendableOrNot, 1, i, dotProduct))
      totalMappingsInClass1 += 1
    else:
      # nonextendable
      # print(f"{extendableOrNot} {dotProduct} (class 2)")
      classifiedMappingResults.append((allMappings[i], extendableOrNot, 2, i, dotProduct))
      totalMappingsInClass2 += 1

  goodClassifierFlag = True
  for mapping, extendability, _class, mapID, dotProduct in classifiedMappingResults:
    if (extendability == "E" and _class == 2):
      print("Bad classifier, extendableMapping in class 2")
      print(mapping.reshape(4,-1), extendability, _class, mapID, dotProduct)
      goodClassifierFlag = False
      break
    if totalMappingsInClass1 < 1:
      print("Bad classifier, no mappings in class 1")
      goodClassifierFlag = False
      break
    if totalMappingsInClass2 < 1:
      print("Bad classifier, no mappings in class 2")
      goodClassifierFlag = False
      break

  if goodClassifierFlag:
    trueClassifiersGood = np.concatenate((trueClassifiersGood, [trueClassifiers[k]]), axis=0)
  else:
    trueClassifiersBad = np.concatenate((trueClassifiersBad, [trueClassifiers[k]]), axis=0)

  print(f"trueClassifiers[{k}]: \n{trueClassifiers[k].reshape(4,-1)}")
  print(f"totalMappingsInClass1: {totalMappingsInClass1}")
  print(f"totalMappingsInClass2: {totalMappingsInClass2}")
  print()

trueClassifiersGood = trueClassifiersGood[1:]
trueClassifiersBad = trueClassifiersBad[1:]

Streaming output truncated to the last 5000 lines.
[[  1.  -2.  -1.  -0.]
 [ -6.  -2.  -8.   0.]
 [ -1.   2.   1.  -0.]
 [ -4.  -6. -10.   0.]]
totalMappingsInClass1: 100000
totalMappingsInClass2: 0

Bad classifier, no mappings in class 2
trueClassifiers[509]: 
[[ 2. -5. -2. -2.]
 [-6.  0. -2. -4.]
 [-2. -1. -3.  0.]
 [-2. -4. -1. -6.]]
totalMappingsInClass1: 100000
totalMappingsInClass2: 0

Bad classifier, extendableMapping in class 2
[[ 0  0  0  0]
 [-2  6  2  2]
 [ 0  0  0  0]
 [-2  6  2  2]] E 2 602 2.0
trueClassifiers[510]: 
[[ 1. -4. -5.  2.]
 [-9.  1. -2. -5.]
 [-2. -1. -2. -1.]
 [-6. -1. -5. -2.]]
totalMappingsInClass1: 99872
totalMappingsInClass2: 128

Bad classifier, no mappings in class 2
trueClassifiers[511]: 
[[ 1. -4. -1. -2.]
 [-5. -1. -1. -5.]
 [-2.  1.  1. -2.]
 [-1. -6. -3. -5.]]
totalMappingsInClass1: 100000
totalMappingsInClass2: 0

trueClassifiers[512]: 
[[ 0. -5. -0. -4.]
 [-2. -2.  2. -6.]
 [-0. -0.  0. -1.]
 [-1. -7.  1. -9.]]
totalMappingsInClass1: 99718
totalM

In [30]:
print(len(trueClassifiersGood))
print(len(trueClassifiersBad))

552
448


In [32]:
for i in range(len(trueClassifiersGood)):
  print(f"trueClassifiersGood[{i}]:")
  print(trueClassifiersGood[i].reshape(4,-1))
  print()

trueClassifiersGood[0]:
[[ -1.  -5.   1.  -6.]
 [ -1.  -2.   1.  -3.]
 [  1.   2.  -1.   3.]
 [ -2.  -8.   2. -12.]]

trueClassifiersGood[1]:
[[ 1. -5. -2. -2.]
 [-3. -2.  0. -6.]
 [-1.  0.  1. -2.]
 [-1. -7. -3. -5.]]

trueClassifiersGood[2]:
[[ 1. -4. -1. -3.]
 [-5. -1.  1. -6.]
 [-1. -0.  0. -1.]
 [-3. -5. -1. -7.]]

trueClassifiersGood[3]:
[[ 1. -5. -2. -2.]
 [-3. -2.  1. -6.]
 [-1. -0.  1. -2.]
 [-1. -7. -2. -6.]]

trueClassifiersGood[4]:
[[ -0.  -7.  -7.   0.]
 [ -0.  -4.  -4.  -0.]
 [  0.  -0.   0.  -0.]
 [ -0. -10. -11.   0.]]

trueClassifiersGood[5]:
[[ 1. -6. -3. -3.]
 [-5.  0. -6.  1.]
 [-1. -0. -1. -1.]
 [-3. -5. -8. -1.]]

trueClassifiersGood[6]:
[[ -1.  -2.  -6.   3.]
 [ -1.  -7.  -3.  -5.]
 [  1.  -1.   3.  -3.]
 [ -3.  -9. -12.   1.]]

trueClassifiersGood[7]:
[[ -2.  -5.  -8.   1.]
 [ -2.  -0.  -3.   1.]
 [  2.   0.   3.  -1.]
 [ -7.  -6. -15.   2.]]

trueClassifiersGood[8]:
[[  4.   3.  10.  -3.]
 [-13.  -8. -29.   8.]
 [ -4.  -3. -10.   3.]
 [ -4.  -3. -10.   2.]]

tr

In [31]:
# Save trueClassifiersGood to a file
np.save('/content/trueClassifiersGood.npy', trueClassifiersGood)
# Save trueClassifiersBad to a file
np.save('/content/trueClassifiersBad.npy', trueClassifiersBad)

In [ ]:
for i in range(len(extendableMappings[0:100])):
  for row in extendableMappings[i]:
      print(row)
  print(f"{i}th extendable")
  print()

In [ ]:
largestMisclassification = 0
largestMisclassificationClassifierIndex = 0

for k in range(len(trueClassifiersBad)):
  totalMappingsInClass1 = 0
  totalMappingsInClass2 = 0
  classifiedMappingResults = []

  for i in range(len(allMappings)):
    dotProduct = np.dot(trueClassifiersBad[k], allMappings[i])
    dotProduct = np.sum(dotProduct)
    dotProductNormalized = dotProduct/(np.linalg.norm(trueClassifiersBad[k]) * np.linalg.norm(allMappings[i]))
    extendableOrNot = "E" if i<len(extendableMappings) else "N"
    if dotProduct <= 0:
      # print(f"{extendableOrNot} {dotProduct} (class 1)")
      classifiedMappingResults.append((allMappings[i], extendableOrNot, 1, i, dotProduct, dotProductNormalized))
      totalMappingsInClass1 += 1
    else:
      # print(f"{extendableOrNot} {dotProduct} (class 2)")
      classifiedMappingResults.append((allMappings[i], extendableOrNot, 2, i, dotProduct, dotProductNormalized))
      totalMappingsInClass2 += 1

  for mapping, extendability, _class, mapID, dotProduct, dotProductNormalized in classifiedMappingResults:
    if (extendability == "E" and _class == 2):
      # print("Bad classifier, extendableMapping in class 2")
      # problemMappings = np.concatenate((problemMappings, [mapping]), axis=0)
      # print(mapping.reshape(4,-1), extendability, _class, mapID, dotProduct, dotProductNormalized)
      if dotProductNormalized > largestMisclassification:
        largestMisclassification = dotProductNormalized
        largestMisclassificationClassifierIndex = k
      # break
    if totalMappingsInClass1 < 1:
      print("Bad classifier, no mappings in class 1")
      break
    if totalMappingsInClass2 < 1:
      print("Bad classifier, no mappings in class 2")
      break

  print(f"trueClassifiersBad[{k}]:")
  print(f"totalMappingsInClass1: {totalMappingsInClass1}")
  print(f"totalMappingsInClass2: {totalMappingsInClass2}")
  print()

print(f"From trueClassifiersBad[{largestMisclassificationClassifierIndex}]")
print(f"Largest misclassification: {largestMisclassification}")


trueClassifiersBad[0]:
totalMappingsInClass1: 99682
totalMappingsInClass2: 318

trueClassifiersBad[1]:
totalMappingsInClass1: 99902
totalMappingsInClass2: 98

trueClassifiersBad[2]:
totalMappingsInClass1: 99979
totalMappingsInClass2: 21

trueClassifiersBad[3]:
totalMappingsInClass1: 97095
totalMappingsInClass2: 2905

trueClassifiersBad[4]:
totalMappingsInClass1: 99797
totalMappingsInClass2: 203

trueClassifiersBad[5]:
totalMappingsInClass1: 99814
totalMappingsInClass2: 186

trueClassifiersBad[6]:
totalMappingsInClass1: 99603
totalMappingsInClass2: 397

trueClassifiersBad[7]:
totalMappingsInClass1: 99963
totalMappingsInClass2: 37

trueClassifiersBad[8]:
totalMappingsInClass1: 98694
totalMappingsInClass2: 1306

trueClassifiersBad[9]:
totalMappingsInClass1: 99952
totalMappingsInClass2: 48

trueClassifiersBad[10]:
totalMappingsInClass1: 99282
totalMappingsInClass2: 718

trueClassifiersBad[11]:
totalMappingsInClass1: 99976
totalMappingsInClass2: 24

trueClassifiersBad[12]:
totalMappingsInCl

In [2]:
import numpy as np
# Load the saved trueClassifiersGood
trueClassifiersGood = np.load('/content/trueClassifiersGood.npy')
print(f"len(trueClassifiersGood): {len(trueClassifiersGood)}")

len(trueClassifiersGood): 61


In [ ]:
for e in extendableMappings:
  if e[0][0] > e[0][1]:
    print("BAD CLASSIFICATION")
    print(e)
    break
  if e[3][0] > e[3][1]:
    print("BAD CLASSIFICATION")
    print(e)
    break

BAD CLASSIFICATION
[[-2  9  4  3]
 [ 9  0  9  0]
 [-1  8  6  1]
 [ 8  1  7  2]]
